<a href="https://colab.research.google.com/github/hasdasda/HoctheoPhamDinhKhanh/blob/main/BaitapcoHuongipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive
%cd VNPT
%cd VNPT_01

Mounted at /content/drive
/content/drive/My Drive
/content/drive/.shortcut-targets-by-id/1UnC-iLs86hXm-MhLt4GMg_ua_MKNBL4I/VNPT
/content/drive/.shortcut-targets-by-id/1UnC-iLs86hXm-MhLt4GMg_ua_MKNBL4I/VNPT/VNPT_01


In [3]:
#Bước 1: Import thư viện
import pandas as pd
import numpy as np
from math import floor
from sklearn.preprocessing import LabelEncoder

In [5]:
#Bước 2: Đọc file đầu vào
#Nếu file input là file excel thì thay read_csv bằng read_excel
#header chỉ dòng chứa tên các features, index_col chỉ cột chứa số thứ tự các mẫu đầu vào
#Nếu có header/index đặtt giá trị biến = 0, nếu không đặt = None
data = pd.read_csv('UNSW_2018_IoT_Botnet_Dataset_1.csv', header=0, index_col=None)

<ipython-input-5-86e761833f0b>:5: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('UNSW_2018_IoT_Botnet_Dataset_1.csv', header=0, index_col=None)


In [6]:
data

,1,1526344121.188091,e,arp,192.168.100.1,Unnamed: 5,192.168.100.3,Unnamed: 7,4,240,...,2,2.1,120,120.1,0.002508,0.000836,0.000836.1,0,Normal,Normal.1
0,2,1.526344e+09,e,tcp,192.168.100.7,139,192.168.100.4,36390,10,680,...,5,5,350,330,0.006190,0.002751,0.002751,0,Normal,Normal
1,3,1.526344e+09,e,udp,192.168.100.149,51838,27.124.125.250,123,2,180,...,1,1,90,90,20.590960,0.000000,0.000000,0,Normal,Normal
2,4,1.526344e+09,e,arp,192.168.100.4,NaN,192.168.100.7,NaN,10,510,...,5,5,210,300,0.006189,0.002751,0.002751,0,Normal,Normal
3,5,1.526344e+09,e,udp,192.168.100.27,58999,192.168.100.1,53,4,630,...,2,2,174,456,0.005264,0.001755,0.001755,0,Normal,Normal
4,6,1.526344e+09,e,arp,192.168.100.1,NaN,192.168.100.27,NaN,2,120,...,1,1,60,60,2724.795654,0.000000,0.000000,0,Normal,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,999996,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,12020,2,120,...,1,1,60,60,23.854963,0.000000,0.000000,1,Reconnaissance,Service_Scan
999995,999997,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,60532,2,120,...,1,1,60,60,23.854963,0.000000,0.000000,1,Reconnaissance,Service_Scan
999996,999998,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,52144,2,120,...,1,1,60,60,23.862362,0.000000,0.000000,1,Reconnaissance,Service_Scan
999997,999999,1.526949e+09,e,tcp,192.168.100.150,49731,192.168.100.3,58105,2,120,...,1,1,60,60,23.854963,0.000000,0.000000,1,Reconnaissance,Service_Scan
